# Compare output with processed data provided by TTC

In [8]:
from psycopg2 import connect
import configparser
%matplotlib inline
import numpy as np
import pandas as pd
import pandas.io.sql as pandasql
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import matplotlib.ticker as ticker
import folium

CONFIG = configparser.ConfigParser()
CONFIG.read('db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

from IPython.display import HTML

def print_table(sql, con):
    return HTML(pandasql.read_sql(sql, con).to_html(index=False))

## Explore the data provided by the TTC 

There are 93 combinations of distinct from stops, to stops, and directions that we have time data from. 

In [13]:
sql_avg = '''
SELECT DISTINCT fromstopname, tostopname, from_stop_id, to_stop_id, directionid, COUNT(*) cnt, routenumber, 
AVG(EXTRACT(EPOCH FROM (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(toarrstoptime, 'HH24:MI:SS'))::timestamp
- (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(fromstopdepaturetime, 'HH24:MI:SS'))::timestamp)  ) /60
AS time_diff_minutes
FROM section_runs
WHERE routenumber = 504
GROUP BY fromstopname, tostopname, directionid,from_stop_id, to_stop_id, routenumber
ORDER BY time_diff_minutes; 
'''

pandasql.read_sql(sql_avg, con)

,fromstopname,tostopname,from_stop_id,to_stop_id,directionid,cnt,routenumber,time_diff_minutes
0,BROADVIEW STATION AT BAY 6,DUNDAS WEST STATION AT BAY 4,13050,13209,0,2,504,-940.666667
1,KING ST EAST AT JARVIS ST,BROADVIEW STATION AT BAY 6,1897,13050,0,4,504,-344.833333
2,DUNDAS WEST STATION AT BAY 4,KING ST WEST AT BATHURST ST,13209,2253,0,9,504,-137.592593
3,KING ST EAST AT PARLIAMENT ST,KING ST EAST AT PARLIAMENT ST,8207,8207,0,98,504,-4.095238
4,KING ST EAST AT JARVIS ST,KING ST EAST AT PARLIAMENT ST,1897,8207,0,1696,504,-0.682390
5,KING ST EAST AT JARVIS ST,KING ST EAST AT YONGE ST,5334,7034,1,1631,504,0.104016
6,KING ST EAST AT YONGE ST,KING ST WEST AT YONGE ST,7034,3070,0,2,504,0.333333
7,KING ST WEST AT BATHURST ST,KING ST WEST AT BATHURST ST,2253,2253,0,2,504,0.333333
8,KING ST WEST AT YONGE ST,KING ST EAST AT YONGE ST,3070,7034,1,2,504,0.333333
9,KING ST WEST AT UNIVERSITY AVE,KING ST WEST AT SPADINA AVE,3357,436,1,1604,504,0.462594


Look at all the stops that segments start at. 

In [2]:
sql_from_stop = '''
SELECT DISTINCT fromstopname, from_stop_id
FROM section_runs
order by from_stop_id; 
'''

pandasql.read_sql(sql_from_stop, con)

,fromstopname,from_stop_id
0,KING ST WEST AT SPADINA AVE,436
1,KING ST EAST AT PARLIAMENT ST,1389
2,KING ST WEST AT UNIVERSITY AVE,1845
3,KING ST EAST AT JARVIS ST,1897
4,KING ST WEST AT BATHURST ST,2253
5,KING ST WEST AT YONGE ST,3070
6,KING ST WEST AT UNIVERSITY AVE,3357
7,KING ST WEST AT DUFFERIN ST,4341
8,KING ST WEST AT DUFFERIN ST,4568
9,KING ST WEST AT SPADINA AVE,4748


Look at all stops where the segments end. 

In [3]:
sql_to_stop = '''
SELECT DISTINCT tostopname, to_stop_id
FROM section_runs
order by to_stop_id; 
'''

pandasql.read_sql(sql_to_stop, con)

,tostopname,to_stop_id
0,KING ST WEST AT SPADINA AVE,436
1,KING ST EAST AT PARLIAMENT ST,1389
2,DISTILLERY LOOP,1845
3,KING ST WEST AT UNIVERSITY AVE,1845
4,KING ST EAST AT JARVIS ST,1897
5,KING ST WEST AT BATHURST ST,2253
6,KING ST WEST AT YONGE ST,3070
7,KING ST WEST AT UNIVERSITY AVE,3357
8,KING ST WEST AT DUFFERIN ST,4341
9,KING ST WEST AT DUFFERIN ST,4568


## Compare averages between TTC and our processed data

Now calculate the average time between this segments from our CIS data processing.

Filtering for where arrival_time at the last stop in the segment is greater than departure time of the first stop in the segment means positive and negative values will not get filtered together. However, it also means that some values with the opposite direction are filtered out. 

In [11]:
sql_cis_avg = '''
WITH to_stop_table AS (
SELECT t.stop_id to_stop, trip_id, direction_id, arrival_time
FROM crosic.cis_504_11192017_11252017_tripids t
WHERE t.stop_id IN (SELECT to_stop_id FROM crosic.section_runs)
),

from_stop_table AS (
SELECT t.stop_id from_stop, trip_id, direction_id, departure_time
FROM crosic.cis_504_11192017_11252017_tripids t 
WHERE t.stop_id IN (SELECT from_stop_id FROM crosic.section_runs)
), 

stops AS (
SELECT to_stop, from_stop, t.direction_id, t.trip_id, arrival_time, departure_time 
FROM from_stop_table f JOIN to_stop_table t ON f.trip_id = t.trip_id and f.direction_id = t.direction_id
),

ttc_cis AS (
SELECT DISTINCT fromstopname, tostopname, directionid, COUNT(*) cnt, to_stop_id, from_stop_id, routenumber,
AVG(EXTRACT(EPOCH FROM (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(toarrstoptime, 'HH24:MI:SS'))::timestamp
- (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(fromstopdepaturetime, 'HH24:MI:SS'))::timestamp)  ) /60
AS time_diff_minutes_ttc
FROM section_runs
WHERE routenumber = 504 and toarrstoptime > fromstopdepaturetime
GROUP BY fromstopname, tostopname, to_stop_id, from_stop_id, directionid, routenumber
ORDER BY time_diff_minutes_ttc
)

SELECT routenumber, to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, cnt cnt_ttc, time_diff_minutes_ttc, COUNT(*) cnt_our_cis,

AVG(EXTRACT(EPOCH FROM arrival_time - departure_time)) /60
AS time_diff_minutes_our_cis


FROM stops s JOIN ttc_cis ttc ON s.to_stop = ttc.to_stop_id AND s.from_stop = ttc.from_stop_id 

WHERE s.direction_id = ttc.directionid and ttc.routenumber = 504 and arrival_time > departure_time
GROUP BY to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, ttc.cnt, time_diff_minutes_ttc, routenumber
ORDER BY time_diff_minutes_ttc, time_diff_minutes_our_cis; 

'''



pandasql.read_sql(sql_cis_avg, con)

,routenumber,to_stop_id,from_stop_id,fromstopname,tostopname,directionid,cnt_ttc,time_diff_minutes_ttc,cnt_our_cis,time_diff_minutes_our_cis
0,504,2253,2253,KING ST WEST AT BATHURST ST,KING ST WEST AT BATHURST ST,0,2,0.333333,53,1.094340
1,504,1389,1389,KING ST EAST AT PARLIAMENT ST,KING ST EAST AT PARLIAMENT ST,1,2,0.500000,6,9.888889
2,504,1897,1897,KING ST EAST AT JARVIS ST,KING ST EAST AT JARVIS ST,0,4,1.083333,15,2.555556
3,504,7211,436,KING ST WEST AT SPADINA AVE,KING ST WEST AT BATHURST ST,1,1686,2.059134,4581,3.269897
4,504,8207,1897,KING ST EAST AT JARVIS ST,KING ST EAST AT PARLIAMENT ST,0,1692,2.715721,5048,2.851760
5,504,7034,5334,KING ST EAST AT JARVIS ST,KING ST EAST AT YONGE ST,1,1628,2.753675,5218,2.517056
6,504,1897,3070,KING ST WEST AT YONGE ST,KING ST EAST AT JARVIS ST,0,1561,3.083280,3780,3.322227
7,504,3070,1845,KING ST WEST AT UNIVERSITY AVE,KING ST WEST AT YONGE ST,0,1562,3.221084,3739,2.574735
8,504,3357,7034,KING ST EAST AT YONGE ST,KING ST WEST AT UNIVERSITY AVE,1,1588,3.408270,4975,2.760121
9,504,4748,2253,KING ST WEST AT BATHURST ST,KING ST WEST AT SPADINA AVE,0,1695,3.450157,4924,3.342134


There is not a lot of data for when the departure time is greater than the arrival time. 

In [12]:
sql_cis_avg = '''
WITH to_stop_table AS (
SELECT t.stop_id to_stop, trip_id, direction_id, arrival_time
FROM crosic.cis_504_11192017_11252017_tripids t
WHERE t.stop_id IN (SELECT to_stop_id FROM crosic.section_runs)
),

from_stop_table AS (
SELECT t.stop_id from_stop, trip_id, direction_id, departure_time
FROM crosic.cis_504_11192017_11252017_tripids t 
WHERE t.stop_id IN (SELECT from_stop_id FROM crosic.section_runs)
), 

stops AS (
SELECT to_stop, from_stop, t.direction_id, t.trip_id, arrival_time, departure_time 
FROM from_stop_table f JOIN to_stop_table t ON f.trip_id = t.trip_id and f.direction_id = t.direction_id
),

ttc_cis AS (
SELECT DISTINCT fromstopname, tostopname, directionid, COUNT(*) cnt, to_stop_id, from_stop_id, routenumber,
AVG(EXTRACT(EPOCH FROM (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(fromstopdepaturetime, 'HH24:MI:SS'))::timestamp
- (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(toarrstoptime, 'HH24:MI:SS'))::timestamp)  ) /60
AS time_diff_minutes_ttc
FROM section_runs
WHERE routenumber = 504 and toarrstoptime < fromstopdepaturetime
GROUP BY fromstopname, tostopname, to_stop_id, from_stop_id, directionid, routenumber
ORDER BY time_diff_minutes_ttc
)

SELECT routenumber, to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, cnt cnt_ttc, time_diff_minutes_ttc, COUNT(*) cnt_our_cis,

AVG(EXTRACT(EPOCH FROM departure_time -  arrival_time)) /60
AS time_diff_minutes_our_cis


FROM stops s JOIN ttc_cis ttc ON s.to_stop = ttc.to_stop_id AND s.from_stop = ttc.from_stop_id 

WHERE s.direction_id = ttc.directionid and ttc.routenumber = 504 and arrival_time < departure_time
GROUP BY to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, ttc.cnt, time_diff_minutes_ttc, routenumber
ORDER BY time_diff_minutes_ttc, time_diff_minutes_our_cis; 

'''



pandasql.read_sql(sql_cis_avg, con)

,routenumber,to_stop_id,from_stop_id,fromstopname,tostopname,directionid,cnt_ttc,time_diff_minutes_ttc,cnt_our_cis,time_diff_minutes_our_cis
0,504,13209,13050,BROADVIEW STATION AT BAY 6,DUNDAS WEST STATION AT BAY 4,0,2,940.666667,251,83.767596
1,504,8207,8207,KING ST EAST AT PARLIAMENT ST,KING ST EAST AT PARLIAMENT ST,0,2,1420.666667,2571,0.646694
2,504,4341,4341,KING ST WEST AT DUFFERIN ST,KING ST WEST AT DUFFERIN ST,1,2,1422.833333,2735,1.618013
3,504,4341,7211,KING ST WEST AT BATHURST ST,KING ST WEST AT DUFFERIN ST,1,6,1432.611111,1,1397.666667
4,504,2253,4568,KING ST WEST AT DUFFERIN ST,KING ST WEST AT BATHURST ST,0,5,1433.400000,1,304.000000


Filter out rows where the count for either our data or the ttc data is less than 10 because the averages for these time differences could be easily skewed by outliers. 

Overall, most segments have similar travel times. In the last three rows in the table below, there seems to be very big differences in the travel times accross the segments. However, the last two rows have a count of under 20 for the ttc processed CIS data, so there could be some outliers that are skewing the results. 

In [15]:
sql_cis_avg  = '''
WITH to_stop_table AS (
SELECT t.stop_id to_stop, trip_id, direction_id, arrival_time
FROM crosic.cis_504_11192017_11252017_tripids t
WHERE t.stop_id IN (SELECT to_stop_id FROM crosic.section_runs)
),

from_stop_table AS (
SELECT t.stop_id from_stop, trip_id, direction_id, departure_time
FROM crosic.cis_504_11192017_11252017_tripids t 
WHERE t.stop_id IN (SELECT from_stop_id FROM crosic.section_runs)
), 

stops AS (
SELECT to_stop, from_stop, t.direction_id, t.trip_id, arrival_time, departure_time 
FROM from_stop_table f JOIN to_stop_table t ON f.trip_id = t.trip_id and f.direction_id = t.direction_id
),

ttc_cis AS (
SELECT DISTINCT fromstopname, tostopname, directionid, COUNT(*) cnt, to_stop_id, from_stop_id, routenumber,
AVG(EXTRACT(EPOCH FROM (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(toarrstoptime, 'HH24:MI:SS'))::timestamp
- (to_char(journeydate, 'YYYY-MM-DD') || ' ' || to_char(fromstopdepaturetime, 'HH24:MI:SS'))::timestamp)  ) /60
AS time_diff_minutes_ttc
FROM section_runs
WHERE routenumber = 504 and toarrstoptime > fromstopdepaturetime
GROUP BY fromstopname, tostopname, to_stop_id, from_stop_id, directionid, routenumber
ORDER BY time_diff_minutes_ttc
),

output AS (
SELECT routenumber, to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, cnt cnt_ttc, time_diff_minutes_ttc, 
COUNT(*) cnt_our_cis,

AVG(EXTRACT(EPOCH FROM arrival_time - departure_time)) /60
AS time_diff_minutes_our_cis


FROM stops s JOIN ttc_cis ttc ON s.to_stop = ttc.to_stop_id AND s.from_stop = ttc.from_stop_id 

WHERE s.direction_id = ttc.directionid and ttc.routenumber = 504 and arrival_time > departure_time
GROUP BY to_stop_id, from_stop_id,  fromstopname, tostopname, directionid, ttc.cnt, time_diff_minutes_ttc, routenumber
ORDER BY time_diff_minutes_ttc, time_diff_minutes_our_cis
) 

SELECT * 
FROM output
WHERE cnt_ttc > 10 AND cnt_our_cis > 10; 

'''



pandasql.read_sql(sql_cis_avg, con)

,routenumber,to_stop_id,from_stop_id,fromstopname,tostopname,directionid,cnt_ttc,time_diff_minutes_ttc,cnt_our_cis,time_diff_minutes_our_cis
0,504,7211,436,KING ST WEST AT SPADINA AVE,KING ST WEST AT BATHURST ST,1,1686,2.059134,4581,3.269897
1,504,8207,1897,KING ST EAST AT JARVIS ST,KING ST EAST AT PARLIAMENT ST,0,1692,2.715721,5048,2.851760
2,504,7034,5334,KING ST EAST AT JARVIS ST,KING ST EAST AT YONGE ST,1,1628,2.753675,5218,2.517056
3,504,1897,3070,KING ST WEST AT YONGE ST,KING ST EAST AT JARVIS ST,0,1561,3.083280,3780,3.322227
4,504,3070,1845,KING ST WEST AT UNIVERSITY AVE,KING ST WEST AT YONGE ST,0,1562,3.221084,3739,2.574735
5,504,3357,7034,KING ST EAST AT YONGE ST,KING ST WEST AT UNIVERSITY AVE,1,1588,3.408270,4975,2.760121
6,504,4748,2253,KING ST WEST AT BATHURST ST,KING ST WEST AT SPADINA AVE,0,1695,3.450157,4924,3.342134
7,504,5334,1389,KING ST EAST AT PARLIAMENT ST,KING ST EAST AT JARVIS ST,1,1634,3.471440,4678,3.120418
8,504,1845,4748,KING ST WEST AT SPADINA AVE,KING ST WEST AT UNIVERSITY AVE,0,1642,4.447818,4682,3.807661
9,504,436,3357,KING ST WEST AT UNIVERSITY AVE,KING ST WEST AT SPADINA AVE,1,1599,4.953096,4292,3.770329


More analysis ideas: remove weekends, look at 'rush hours', remove 'rush hour', look at the times when the processed data is availiable for each segement

In [6]:
con.close()